In [60]:
from pathlib import Path
import sys
import pandas as pd
import altair as alt

In [61]:
ROOT_DIR = Path().resolve().parent
SRC_DIR = ROOT_DIR / 'src'
DATA_DIR = ROOT_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'

sys.path.append(str(SRC_DIR))
from reading_data import INTERIM_DIR

df_municipios = pd.read_parquet(RAW_DIR / 'data_municipios.parquet')
df_policia = pd.read_parquet(RAW_DIR / 'data_policia.parquet')
df_violencia_mortes = pd.read_csv(RAW_DIR / "data_violencia_mortes.csv.gz")

In [62]:
# Selecting columns to create df with police violence
colunas_importantes = [
    "ano",
    "sigla_uf",
    "id_municipio",
    "grupo",
]

# Selecionando mortes por violência policial
df_mortes_interv_policiais = df_violencia_mortes.filter(like='mortes_intervencao_policial', axis=1)
# Selecionando colunas com informações
df_informacoes = df_violencia_mortes[colunas_importantes]
# Juntando os dois dataframes
df_mortes_interv_policiais = pd.concat([df_informacoes, df_mortes_interv_policiais], axis=1)

In [63]:
df_final = pd.read_parquet(INTERIM_DIR / 'data_final.parquet')

In [64]:
# juntando com a tabela de informações de município
df_municipios_join = df_municipios[['id_municipio', 'nome']]

# Merging the dataframes
df_final = df_municipios_join.merge(df_mortes_interv_policiais, on='id_municipio', how='inner', suffixes=("_left", '_right'))

# Dropping columns
df_final = df_final.drop(columns=['id_municipio'])

# renaming columns
rename_dict = {
    'quantidade_mortes_intervencao_policial': 'qtd_interv_pol',
    'quantidade_mortes_intervencao_policial_civil_em_servico': 'qtd_pol_civ_serv',
    'quantidade_mortes_intervencao_policial_militar_em_servico': 'qtd_pol_mil_ser',
    'quantidade_mortes_intervencao_policial_civil_fora_de_servico': 'qtd_pol_civ_fora',
    'quantidade_mortes_intervencao_policial_militar_fora_de_servico': 'qtd_pol_mil_fora',
    'nome': 'municipio',
}

df_final = df_final.rename(columns=rename_dict)

# Adding states
estados_para_regiao = {
    # Região Norte
    'AC': 'Norte',
    'AP': 'Norte',
    'AM': 'Norte',
    'PA': 'Norte',
    'RO': 'Norte',
    'RR': 'Norte',
    'TO': 'Norte',
    
    # Região Nordeste
    'AL': 'Nordeste',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'MA': 'Nordeste',
    'PB': 'Nordeste',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'RN': 'Nordeste',
    'SE': 'Nordeste',
    
    # Região Centro-Oeste
    'DF': 'Centro-Oeste',
    'GO': 'Centro-Oeste',
    'MT': 'Centro-Oeste',
    'MS': 'Centro-Oeste',
    
    # Região Sudeste
    'ES': 'Sudeste',
    'MG': 'Sudeste',
    'RJ': 'Sudeste',
    'SP': 'Sudeste',
    
    # Região Sul
    'PR': 'Sul',
    'RS': 'Sul',
    'SC': 'Sul'
}

df_final['regiao'] = df_final['sigla_uf'].map(estados_para_regiao)

df_final

,municipio,ano,sigla_uf,grupo,qtd_interv_pol,qtd_pol_civ_serv,qtd_pol_mil_ser,qtd_pol_civ_fora,qtd_pol_mil_fora,regiao
0,Porto Velho,2016,RO,Grupo 3,6.0,0.0,3.0,0.0,3.0,Norte
1,Porto Velho,2017,RO,Grupo 3,4.0,NaN,3.0,0.0,1.0,Norte
2,Porto Velho,2018,RO,NaN,2.0,NaN,NaN,NaN,NaN,Norte
3,Porto Velho,2019,RO,NaN,7.0,NaN,NaN,NaN,NaN,Norte
4,Porto Velho,2020,RO,Grupo 3,3.0,NaN,NaN,NaN,NaN,Norte
...,...,...,...,...,...,...,...,...,...,...
157,Brasília,2017,DF,Grupo 2,7.0,0.0,2.0,0.0,5.0,Centro-Oeste
158,Brasília,2018,DF,NaN,5.0,NaN,NaN,NaN,NaN,Centro-Oeste
159,Brasília,2019,DF,NaN,8.0,NaN,NaN,NaN,NaN,Centro-Oeste
160,Brasília,2020,DF,Grupo 1,11.0,NaN,NaN,NaN,NaN,Centro-Oeste


### Dados
- 27 municípios
- 6 anos

Algmumas visualizações:
- Mortes ao longo dos anos
  - Dividindo por policial civil e militar

Black hat:
- Selecionar apenas as cidades onde houve uma diminuição das mortes, fazendo uma agregação que favoreça essa diminuição

White hat:
- Juntar os dados para demonstrar que a maioria das mortes são de pessoas pretas

### Dataframe com populações das cidades em 2022

In [65]:
# Reading population of cities in 2022
# URL of the Wikipedia page containing the table
url = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_popula%C3%A7%C3%A3o_(2022)"
# Read HTML tables into a list of DataFrames
tables = pd.read_html(url)
# Select the desired table (usually the first one)
df_pop_cities = tables[0]

df_pop_cities = df_pop_cities.rename(columns={'Município': 'municipio', 'População': 'pop'})

df_pop_cities = df_pop_cities.drop(columns=['Posição', 'Código IBGE', 'Unidade federativa'])

df_pop_cities.head()

,municipio,pop
0,São Paulo,11 451 999
1,Rio de Janeiro,6 211 223
2,Brasília[nota 1],2 817 381
3,Fortaleza,2 428 708
4,Salvador,2 417 678


### Lendo df_final

In [66]:
df_final = pd.read_parquet(INTERIM_DIR / 'data_final.parquet')
df_final.head(5)

,municipio,ano,sigla_uf,grupo,qtd_interv_pol,qtd_pol_civ_serv,qtd_pol_mil_ser,qtd_pol_civ_fora,qtd_pol_mil_fora,regiao
0,Porto Velho,2016,RO,Grupo 3,6.0,0.0,3.0,0.0,3.0,Norte
1,Porto Velho,2017,RO,Grupo 3,4.0,NaN,3.0,0.0,1.0,Norte
2,Porto Velho,2018,RO,None,2.0,NaN,NaN,NaN,NaN,Norte
3,Porto Velho,2019,RO,None,7.0,NaN,NaN,NaN,NaN,Norte
4,Porto Velho,2020,RO,Grupo 3,3.0,NaN,NaN,NaN,NaN,Norte


In [67]:
# Filling NaN values in qtd_interv_pol
display(df_final[pd.isna(df_final['qtd_interv_pol'])])

# Calculating mean values of death for each city
mean_deaths_cities = df_final.groupby('municipio')['qtd_interv_pol'].transform('mean')
df_final['qtd_interv_pol'] = df_final['qtd_interv_pol'].fillna(mean_deaths_cities)

display(df_final[pd.isna(df_final['qtd_interv_pol'])])

,municipio,ano,sigla_uf,grupo,qtd_interv_pol,qtd_pol_civ_serv,qtd_pol_mil_ser,qtd_pol_civ_fora,qtd_pol_mil_fora,regiao
8,Rio Branco,2018,AC,None,NaN,NaN,NaN,NaN,NaN,Norte
18,Boa Vista,2016,RR,Grupo 4,NaN,NaN,NaN,NaN,NaN,Norte
80,Maceió,2018,AL,None,NaN,NaN,NaN,NaN,NaN,Nordeste
144,Cuiabá,2016,MT,Grupo 1,NaN,0.0,NaN,0.0,NaN,Centro-Oeste
152,Goiânia,2018,GO,None,NaN,NaN,NaN,NaN,NaN,Centro-Oeste


,municipio,ano,sigla_uf,grupo,qtd_interv_pol,qtd_pol_civ_serv,qtd_pol_mil_ser,qtd_pol_civ_fora,qtd_pol_mil_fora,regiao


In [68]:
df_aggregated = df_final.groupby(['ano', 'regiao'], as_index=False)['qtd_interv_pol'].sum()
df_aggregated.head(5)

,ano,regiao,qtd_interv_pol
0,2016,Centro-Oeste,137.2
1,2016,Nordeste,311.0
2,2016,Norte,160.6
3,2016,Sudeste,966.0
4,2016,Sul,125.0


In [69]:
# Create the line plot
line_plot = alt.Chart(df_aggregated).mark_line(
    strokeWidth=3,  # Thicker lines for readability
    interpolate='monotone'  # Smooth lines
).encode(
    x=alt.X('ano:O', title='Ano', axis=alt.Axis(labelAngle=0)),  # Treat ano as ordinal
    y=alt.Y('sum(qtd_interv_pol):Q', title='Total de mortes por intervenção policial'),
    color=alt.Color('regiao:N', legend=alt.Legend(title="Regiao"), 
                    scale=alt.Scale(scheme='category10')),  # Color by region
    tooltip=['regiao', 'ano', 'sum(qtd_interv_pol)']  # Optional tooltips
).properties(
    width=600,
    height=400,
    title='Mortes por intervanção policial (2016-2020)'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_legend(
    titleFontSize=12,
    labelFontSize=11
)

line_plot

alt.Chart(...)

### Gráfico selecionando apenas cidades com tendência de descida

In [70]:
from scipy.stats import linregress

def calculate_trend(group):
    slope, _, _, _, _ = linregress(group['ano'], group['qtd_interv_pol'])
    return pd.Series({'Slope': slope, 'First_Year_Deaths': group['qtd_interv_pol'].iloc[0], 'Last_Year_Deaths': group['qtd_interv_pol'].iloc[-1]})

# Group by city and calculate trends
city_trends = df_final.groupby('municipio')[['ano', 'qtd_interv_pol']].apply(calculate_trend).reset_index()
city_trends.head(5)

,municipio,Slope,First_Year_Deaths,Last_Year_Deaths
0,Aracaju,3.685714,21.0,40.0
1,Belo Horizonte,-1.485714,15.0,14.0
2,Belém,6.600000,39.0,77.0
3,Boa Vista,1.085714,6.6,9.0
4,Brasília,0.571429,7.0,8.0


In [71]:
# Selecting only cities with negative slope
negative_slope_cities = city_trends[city_trends['Slope'] < 0]
negative_slope_cities = list(negative_slope_cities['municipio'])
print("negative_slope_cities")
print(negative_slope_cities)

# Filtering relevant cities
df_pop_cities = df_pop_cities[df_pop_cities['municipio'].isin(negative_slope_cities)].drop_duplicates(subset='municipio', keep='last')
df_less_violence_cities = df_final.loc[df_final['municipio'].isin(negative_slope_cities)]

df_pop_cities

# Adding populations of the cities
df_pop_cities_negative = df_pop_cities.loc[df_pop_cities['municipio'].isin(negative_slope_cities)]
df_less_violence_cities = df_less_violence_cities.merge(df_pop_cities_negative, on='municipio', how='left')

# Adjusting numbers of deaths
# Stardadizing dtypes
# Removing spaces and special characters

df_less_violence_cities['pop'] = df_less_violence_cities['pop'].str.replace(r'[^\d.]', '', regex=True).astype(float)
# df_less_violence_cities['qtd_interv_pol'] = df_less_violence_cities['qtd_interv_pol'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# df_less_violence_cities['pop'] = df_less_violence_cities['pop'].astype(float)
df_less_violence_cities['adjusted_deaths'] = df_less_violence_cities['qtd_interv_pol'] / df_less_violence_cities['pop'] * 1e6

df_less_violence_cities

negative_slope_cities
['Belo Horizonte', 'Campo Grande', 'Fortaleza', 'Maceió', 'Porto Alegre', 'Recife', 'Rio Branco', 'Rio de Janeiro', 'São Luís', 'São Paulo']


,municipio,ano,sigla_uf,grupo,qtd_interv_pol,qtd_pol_civ_serv,qtd_pol_mil_ser,qtd_pol_civ_fora,qtd_pol_mil_fora,regiao,pop,adjusted_deaths
0,Rio Branco,2016,AC,Grupo 2,21.0,2.0,11.0,1.0,7.0,Norte,4535.0,4630.650496
1,Rio Branco,2017,AC,Grupo 2,30.0,0.0,25.0,1.0,4.0,Norte,4535.0,6615.214994
2,Rio Branco,2018,AC,None,17.0,NaN,NaN,NaN,NaN,Norte,4535.0,3748.621830
3,Rio Branco,2019,AC,None,12.0,NaN,NaN,NaN,NaN,Norte,4535.0,2646.085998
4,Rio Branco,2020,AC,Grupo 3,16.0,NaN,NaN,NaN,NaN,Norte,4535.0,3528.114664
5,Rio Branco,2021,AC,Grupo 3,6.0,NaN,NaN,NaN,NaN,Norte,4535.0,1323.042999
6,São Luís,2016,MA,Grupo 1,27.0,0.0,27.0,0.0,0.0,Nordeste,7315.0,3691.045796
7,São Luís,2017,MA,Grupo 1,34.0,0.0,34.0,NaN,NaN,Nordeste,7315.0,4647.983595
8,São Luís,2018,MA,None,21.0,NaN,NaN,NaN,NaN,Nordeste,7315.0,2870.813397
9,São Luís,2019,MA,None,33.0,NaN,NaN,NaN,NaN,Nordeste,7315.0,4511.278195


In [72]:
# Create the line plot
line_plot = alt.Chart(df_less_violence_cities).mark_line(
    strokeWidth=3,  # Thicker lines for readability
    interpolate='monotone'  # Smooth lines
).encode(
    x=alt.X('ano:O', title='Ano', axis=alt.Axis(labelAngle=0)),  # Treat ano as ordinal
    y=alt.Y('adjusted_deaths:Q', title='Total de mortes por intervenção policial', scale=alt.Scale(type='log')),
    color=alt.Color('municipio:N', legend=alt.Legend(title="Município"), 
                    scale=alt.Scale(scheme='category10')),  # Color by region
    tooltip=['municipio', 'ano', 'adjusted_deaths']  # Optional tooltips
).properties(
    width=600,
    height=400,
    title='Cidades brasileiras apresentam diminuição nas mortes por violência policial!'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_legend(
    titleFontSize=12,
    labelFontSize=11
)

line_plot

alt.Chart(...)

In [78]:
df_less_violence_cities.head(5)

,municipio,ano,sigla_uf,grupo,qtd_interv_pol,qtd_pol_civ_serv,qtd_pol_mil_ser,qtd_pol_civ_fora,qtd_pol_mil_fora,regiao,pop,adjusted_deaths
0,Rio Branco,2016,AC,Grupo 2,21.0,2.0,11.0,1.0,7.0,Norte,4535.0,4630.650496
1,Rio Branco,2017,AC,Grupo 2,30.0,0.0,25.0,1.0,4.0,Norte,4535.0,6615.214994
2,Rio Branco,2018,AC,None,17.0,NaN,NaN,NaN,NaN,Norte,4535.0,3748.621830
3,Rio Branco,2019,AC,None,12.0,NaN,NaN,NaN,NaN,Norte,4535.0,2646.085998
4,Rio Branco,2020,AC,Grupo 3,16.0,NaN,NaN,NaN,NaN,Norte,4535.0,3528.114664


In [86]:
import numpy as np

# Assuming df_less_violence_cities is your original DataFrame
# Calculate the total deaths per year to create the trend line
total_deaths_per_year = df_less_violence_cities.groupby('ano')['adjusted_deaths'].sum().reset_index()

# Create a linear regression to calculate the trend line
x = total_deaths_per_year['ano'].astype(int)
y = total_deaths_per_year['adjusted_deaths']

# Perform linear regression
coeffs = np.polyfit(x, y, 1)
trend_line = np.poly1d(coeffs)

# Create a DataFrame for the trend line
trend_df = pd.DataFrame({
    'ano': x,
    'trend_deaths': trend_line(x)
})

# Create the trend line chart
trend_line_chart = alt.Chart(trend_df).mark_line(
    color='red', 
    strokeWidth=3, 
    strokeDash=[5, 5]  # Dashed line
).encode(
    x=alt.X('ano:O'),
    y=alt.Y('trend_deaths:Q')
)

# Add text annotation for the trend line
trend_text = alt.Chart(pd.DataFrame({
    'ano': [trend_df['ano'].iloc[-1]],
    'trend_deaths': [trend_df['trend_deaths'].iloc[-1]],
    'text': ['Tendência de queda']
})).mark_text(
    align='right',
    baseline='middle',
    dx=50,
    dy=-50,
    color='red',
    fontSize=16
).encode(
    x=alt.X('ano:O'),
    y=alt.Y('trend_deaths:Q'),
    text='text:N'
)

chart_title = alt.TitleParams(
    'Mortes por intervenções policiais no Brasil',
    subtitle="Cidades brasileiras apresentam diminuição no número de mortes em intervenções policiais"
)

# Create stacked area chart with log scale
stacked_area = alt.Chart(df_less_violence_cities, title=chart_title).mark_area(
    line={'size': 1},
    opacity=0.7,
    interpolate='monotone'
).encode(
    x=alt.X('ano:O', title='Ano',
            axis=alt.Axis(labelAngle=0, labelFontSize=16, titleFontSize=20)),
    
    y=alt.Y('adjusted_deaths:Q',
            title='Total de mortes',
            axis=alt.Axis(labelFontSize=16, titleFontSize=20),
            stack=True
    ),

    color=alt.Color('municipio:N', 
                    legend=alt.Legend(title="Município", labelFontSize=16, titleFontSize=20),
                    scale=alt.Scale(scheme='tableau20')),
    tooltip=['municipio', 'ano', 'adjusted_deaths']
)
# ).properties(
#     width=900,
#     height=600
# ).configure_axis(
#     labelFontSize=12,
#     titleFontSize=14
# ).configure_title(
#     fontSize=20,
#     anchor='middle',
#     subtitleFontSize=16
# )

final_chart = (stacked_area + trend_line_chart + trend_text).properties(
    width=900,
    height=600
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=20,
    anchor='middle',
    subtitleFontSize=16
)

final_chart

alt.LayerChart(...)

## White hat
Gráfico mostrando como policiais matam mais pessoas negras

In [ ]:
df_mortes_interv_policiais.head(5)

,ano,sigla_uf,id_municipio,grupo,quantidade_mortes_intervencao_policial,quantidade_mortes_intervencao_policial_civil_em_servico,quantidade_mortes_intervencao_policial_militar_em_servico,quantidade_mortes_intervencao_policial_civil_fora_de_servico,quantidade_mortes_intervencao_policial_militar_fora_de_servico
0,2016,AL,2704302,Grupo 1,59.0,0.0,57.0,0.0,2.0
1,2016,CE,2304400,Grupo 1,39.0,1.0,23.0,0.0,7.0
2,2016,ES,3205309,Grupo 1,9.0,1.0,6.0,0.0,2.0
3,2016,GO,5208707,Grupo 1,100.0,0.0,76.0,0.0,24.0
4,2016,MA,2111300,Grupo 1,27.0,0.0,27.0,0.0,0.0


In [ ]:
df_policia.columns

Index(['DEPARTAMENTO_CIRCUNSCRICAO', 'SECCIONAL_CIRCUNSCRICAO',
       'MUNICIPIO_CIRCUNSCRICAO', 'DP_CIRCUNSCRICAO ', 'COORPORAÇÃO',
       'SITUAÇÃO', 'ID_DELEGACIA', 'MÊS ESTATISTICA', 'ANO ESTATISTICA',
       'DATAHORA_REGISTRO_BO', 'NUM_BO', 'ANO_BO', 'MUNICIPIO_ELABORACAO',
       'DP_ELABORACAO', 'SEC_ELABORACAO', 'DEP_ELABORACAO', 'DATA_FATO',
       'HORA_FATO', 'DESC_TIPOLOCAL', 'LOGRADOURO', 'NUMERO_LOGRADOURO',
       'LATITUDE', 'LONGITUDE', 'TIPO_PESSOA', 'SEXO_PESSOA', 'IDADE_PESSOA',
       'DATA_NASCIMENTO_PESSOA', 'COR_PELE', 'PROFISSAO', 'NATUREZA_APURADA'],
      dtype='object')

In [ ]:
from typing import List

def copy_and_transform_df_policia():
    copy_df_policia = df_policia.copy()

    # Lowering column names
    copy_df_policia.columns = copy_df_policia.columns.str.lower()
    copy_df_policia.columns

    # Selecting relevant columns to the analysis
    desired_columns = [
        'ano estatistica',
        'ano_bo',
        'municipio_elaboracao',
        'data_fato',
        'data_fato',
        'tipo_pessoa',
        'sexo_pessoa',
        'idade_pessoa',
        'data_nascimento_pessoa',
        'cor_pele',
        'profissao',
    ]
    copy_df_policia = copy_df_policia[desired_columns]

    # Ensurig non null values for the counting
    copy_df_policia['ano_bo'] = copy_df_policia['ano_bo'].fillna(0)

    # Lowering the values in cor_pele for standardizing
    copy_df_policia['cor_pele'] = copy_df_policia['cor_pele'].str.lower()
    
    return copy_df_policia

def groupby_and_count_by_columns(df: pd.DataFrame, cols: List[str]):
    # Getting counts of deaths for each 'cor_pele'
    return (df
        .groupby(cols, observed=True)['ano_bo']
        .size()
        .reset_index(name='contagem')
    )

def remove_rows_from_column(df, drop_list, col):
    drop_rows_index = df.loc[df[col].isin(drop_list)].index
    return df.drop(index=drop_rows_index)

def bin_age(df, age_col, name_group_col):
    # Define age bins and labels
    age_bins = [0, 18, 25, 35, 50, 70, 100]
    age_labels = ['0-18', '19-25', '26-35', '36-50', '51-70', '71+']

    # Create age groups
    df[name_group_col] = pd.cut(
        df[age_col],
        bins=age_bins,
        labels=age_labels,
        right=False  # Includes left edge, excludes right (e.g., 18 is in 0-18)
    )

    return df

def clean_race_column(df: pd.DataFrame):
    drop_rows_list = [
        'ignorada',
        'outros',
        'policia federal',
        'registrado na pf',
    ]
    return remove_rows_from_column(df, drop_rows_list, 'cor_pele')

def clean_sex_column(df: pd.DataFrame):
    # First, removing whitespaces
    df['sexo_pessoa'] = df['sexo_pessoa'].str.replace(' ', '')
    
    # Removing 'Indefinido', 'REGISTRADO NA PF'
    drop_rows_list = [
        'Indefinido',
        'REGISTRADO NA PF',
    ]

    df = remove_rows_from_column(df, drop_rows_list, 'sexo_pessoa')

    df['sexo_pessoa'] = df['sexo_pessoa'].str.lower()

    return df

In [ ]:
copy_df_policia = copy_and_transform_df_policia()
copy_df_policia = groupby_and_count_by_columns(copy_df_policia, ['cor_pele'])
copy_df_policia = clean_race_column(copy_df_policia)

df_mortes_cor_pele = copy_df_policia
df_mortes_cor_pele

,cor_pele,contagem
0,amarela,6
1,branca,3055
4,parda,4812
6,preta,869


In [ ]:
copy_df_policia = copy_and_transform_df_policia()
copy_df_policia = groupby_and_count_by_columns(copy_df_policia, ['sexo_pessoa', 'cor_pele'])
copy_df_policia = clean_race_column(copy_df_policia)
copy_df_policia = clean_sex_column(copy_df_policia)

df_mortes_pele_sexo = copy_df_policia
df_mortes_pele_sexo

,sexo_pessoa,cor_pele,contagem
0,feminino,branca,19
1,feminino,parda,11
2,feminino,preta,4
9,masculino,amarela,6
10,masculino,branca,3021
13,masculino,parda,4788
15,masculino,preta,865


In [88]:
copy_df_policia = copy_and_transform_df_policia()

# Selecting columns
desired_columns = [
    'ano_bo',
    'sexo_pessoa',
    'idade_pessoa',
    'cor_pele',
]
copy_df_policia = copy_df_policia[desired_columns]

# WARNING: removing missing ages to ensure cleaness and no distortion of the data
copy_df_policia = copy_df_policia.dropna(subset='idade_pessoa')
# Removing useless values from idade_pessoa
drop_list = [
    'REGISTRADO NA PF',
    'Policia Federal',
]
copy_df_policia = remove_rows_from_column(copy_df_policia, drop_list, 'idade_pessoa')
# Standardizing age column
copy_df_policia['idade_pessoa'] = copy_df_policia['idade_pessoa'].astype(int)
# Binning age column
copy_df_policia = bin_age(copy_df_policia, 'idade_pessoa', 'faixa_etaria')

copy_df_policia = groupby_and_count_by_columns(copy_df_policia, ['faixa_etaria', 'sexo_pessoa', 'cor_pele'])
copy_df_policia = clean_race_column(copy_df_policia)
copy_df_policia = clean_sex_column(copy_df_policia)

df_idade_sexo_cor_mortes_policia = copy_df_policia

print("Faixas etarias")
print(df_idade_sexo_cor_mortes_policia['faixa_etaria'].unique())
print("Sexos")
print(df_idade_sexo_cor_mortes_policia['sexo_pessoa'].unique())
print("Cores de pele")
print(df_idade_sexo_cor_mortes_policia['cor_pele'].unique())
df_idade_sexo_cor_mortes_policia.sample(n=10)

Faixas etarias
['0-18', '19-25', '26-35', '36-50', '51-70', '71+']
Categories (6, object): ['0-18' < '19-25' < '26-35' < '36-50' < '51-70' < '71+']
Sexos
['feminino' 'masculino']
Cores de pele
['branca' 'parda' 'preta' 'amarela']


,faixa_etaria,sexo_pessoa,cor_pele,contagem
29,36-50,masculino,parda,450
13,19-25,masculino,branca,998
35,71+,feminino,branca,1
8,0-18,masculino,preta,102
34,51-70,masculino,preta,13
23,26-35,masculino,parda,964
3,0-18,masculino,amarela,1
27,36-50,masculino,branca,416
30,36-50,masculino,preta,81
32,51-70,masculino,branca,66


Pela falta de tempo e a necessidade de uma visualização clara, vou remover os valores nulos e expressar isso em algum lugar

In [ ]:
df_idade_sexo_cor_mortes_policia.columns

Index(['faixa_etaria', 'sexo_pessoa', 'cor_pele', 'contagem'], dtype='object')

In [87]:
chart_title = alt.TitleParams(
    'Mortes por violência policial no Brasil',
    subtitle="(Etnia x Idade x Sexo)"
)

chart = alt.Chart(df_idade_sexo_cor_mortes_policia, title = chart_title).mark_bar(
    width=20,
    cornerRadius=3
).encode(
    x=alt.X('sexo_pessoa:N',
            title='Sexo',
            axis=alt.Axis(labelAngle=0, labelFontSize=16, titleFontSize=20)),
    
    y=alt.Y('contagem:Q',
            aggregate='sum',
            title='Número de mortes',
            axis=alt.Axis(labelAngle=0, labelFontSize=16, titleFontSize=20)),
    
    color=alt.Color('cor_pele:N',
                    title='Etnia',
                    legend=alt.Legend(labelFontSize=16, titleFontSize=20, orient='top')),
    
    column=alt.Column('faixa_etaria:N',
                        title='Faixa etária',
                        header=alt.Header(labelFontSize=16, titleFontSize=20)),
    
    xOffset=alt.XOffset('cor_pele:N')  # This creates the side-by-side bars
).properties(
    width=180,  # Adjust per facet
).configure_view(
    strokeOpacity=0
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize = 20,
    anchor='middle',
    subtitleFontSize=16
)

chart

alt.Chart(...)